### Cel modelu globalnego

Cel tego notebooka to zbudowanie stabilnego modelu ML,
który uczy się na wielu szeregach czasowych jednocześnie
(global / segmentowy), z wykorzystaniem wspólnych cech
czasowych oraz cech statycznych (kraj, produkt).

Model nie jest trenowany na pojedynczych produktach ani
pojedynczych krajach, ponieważ w danych każdy produkt
występuje tylko w jednym kraju i reprezentuje jedną serię.

Zamiast tego stosujemy podejście segmentowe:
- grupujemy serie według podobnego zachowania popytu
  (skala + zmienność),
- trenujemy osobny model dla każdego segmentu,
- porównujemy wyniki z najlepszym baseline per seria.

Kluczowe założenia:
- target do treningu: y = log1p(demand),
- ewaluacja na oryginalnej skali (WAPE),
- jeden notebook, jeden pipeline, porównywalne wyniki.

Analiza danych pokazuje, że serie różnią się o rzędy wielkości
skalą popytu oraz poziomem zmienności (CV), ale nie są
intermittent (zero_share ≈ 0).

Trenowanie jednego modelu na wszystkich seriach naraz
prowadziłoby do uśredniania zachowań i utraty jakości.

Z drugiej strony trenowanie osobnego modelu na każdy produkt
nie ma sensu, ponieważ każda seria występuje tylko raz
(product × country = 1 seria).

Rozwiązaniem jest segmentacja serii według zachowania popytu.

Segment A:
- wysoka skala popytu,
- niska / umiarkowana zmienność,
- serie stabilne, najlepiej rokujące dla ML.

Segment A (High volume / Stable):

Serie o dużej skali popytu i niskim CV.
To produkty, dla których:
- ML ma największą szansę poprawić baseline,
- sezonowość i trend są czytelne,
- sygnał dominuje nad szumem.

Ten segment traktujemy jako pierwszy benchmark:
jeżeli ML nie poprawi baseline tutaj,
to nie ma sensu iść dalej.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from lightgbm import LGBMRegressor
import lightgbm as lgb

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 30)

DATA_DIR = Path("data")
FEAT_DIR = DATA_DIR / "features"
BT_DIR = DATA_DIR / "backtesting"

DATA_PATH = FEAT_DIR / "features_level_a.parquet"
BASELINE_PATH = BT_DIR / "baseline_best_per_series.parquet"

DATA_PATH, BASELINE_PATH

(WindowsPath('data/features/features_level_a.parquet'),
 WindowsPath('data/backtesting/baseline_best_per_series.parquet'))

In [16]:
df = pd.read_parquet(DATA_PATH)
baseline_ref = pd.read_parquet(BASELINE_PATH)

df["week_start"] = pd.to_datetime(df["week_start"], errors="coerce")

# w baseline_ref w Twoim pipeline WAPE jest zwykle w kolumnie "wape"
if "baseline_wape" not in baseline_ref.columns:
    if "wape" in baseline_ref.columns:
        baseline_ref = baseline_ref.rename(columns={"wape": "baseline_wape"})

df = df.merge(
    baseline_ref[["country", "sku", "best_baseline", "baseline_wape"]],
    on=["country", "sku"],
    how="left",
)

df.shape, df.columns[:20]

((3578, 43),
 Index(['week_start', 'country', 'sku', 'demand', 'product_name', 'n_dc',
        'is_outlier', 'eligible', 'demand_type', 'zero_share', 'n_weeks_obs',
        'longest_gap_weeks', 'missing_weeks', 'was_missing_week', 'year',
        'week', 'time_idx', 'week_sin', 'week_cos', 'is_q4'],
       dtype='object'))

In [30]:
def wape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.sum(np.abs(y_true))
    return np.nan if denom == 0 else np.sum(np.abs(y_true - y_pred)) / denom

def inv_log1p(x):
    return np.expm1(np.asarray(x, dtype=float))

In [17]:
# Oryginalny target
TARGET_RAW = "demand"

# Target do treningu
TARGET_LOG = "y"

df[TARGET_RAW] = pd.to_numeric(df[TARGET_RAW], errors="coerce")
df[TARGET_LOG] = np.log1p(df[TARGET_RAW].clip(lower=0))  # bezpieczeństwo na wypadek ujemnych/NaN

df[[TARGET_RAW, TARGET_LOG]].describe()

,demand,y
count,3578.000000,3578.000000
mean,2293.084579,6.206251
std,2960.313417,2.442285
min,0.000000,0.000000
25%,174.000000,5.164786
50%,992.000000,6.900731
75%,2988.000000,8.002694
max,17629.000000,9.777357


In [18]:
CAT_FEATURES = ["country", "sku"]

BASE_NUM = [
    "n_dc",
    "week_sin",
    "week_cos",
    "zero_share",
    "ADI",
    "CV2",
    "weeks_since_nonzero",
    "is_zero",
    "is_outlier",
]

KEEP_LAGS = {1, 2, 4, 8, 13}
KEEP_ROLLS = {4, 8, 13}

def pick_feature_cols(columns):
    cols = []
    # bazowe numeryczne
    cols += [c for c in BASE_NUM if c in columns]

    # rollingi: roll_mean_X, roll_std_X
    for c in columns:
        if c.startswith("roll_mean_") or c.startswith("roll_std_"):
            try:
                w = int(c.split("_")[-1])
            except Exception:
                continue
            if w in KEEP_ROLLS:
                cols.append(c)

    # lagi: lag_X (jeśli występują w tym pliku cech)
    for c in columns:
        if c.startswith("lag_"):
            try:
                l_ = int(c.split("_")[-1])
            except Exception:
                continue
            if l_ in KEEP_LAGS:
                cols.append(c)

    # unikamy duplikatów i stabilizujemy kolejność
    cols = sorted(set(cols), key=lambda x: (x not in BASE_NUM, x))
    return cols

NUM_FEATURES = pick_feature_cols(df.columns)

len(NUM_FEATURES), NUM_FEATURES[:30]

(18,
 ['is_outlier',
  'is_zero',
  'n_dc',
  'week_cos',
  'week_sin',
  'weeks_since_nonzero',
  'zero_share',
  'lag_1',
  'lag_13',
  'lag_2',
  'lag_4',
  'lag_8',
  'roll_mean_13',
  'roll_mean_4',
  'roll_mean_8',
  'roll_std_13',
  'roll_std_4',
  'roll_std_8'])

In [19]:
def coerce_flag_col(frame, col):
    if col not in frame.columns:
        return frame
    s = frame[col]
    if s.dtype == "O":
        s = (
            s.astype(str)
            .str.strip()
            .str.lower()
            .replace({"true": 1, "false": 0, "nan": np.nan, "none": np.nan, "": np.nan})
        )
    frame[col] = pd.to_numeric(s, errors="coerce").fillna(0).astype("int8")
    return frame

for c in ["is_outlier", "is_zero"]:
    df = coerce_flag_col(df, c)

needed_cols = ["week_start"] + CAT_FEATURES + ["product_name"] + NUM_FEATURES + [TARGET_RAW, TARGET_LOG, "baseline_wape"]
needed_cols = [c for c in needed_cols if c in df.columns]

model_df = df[needed_cols].dropna(subset=["week_start", TARGET_RAW, TARGET_LOG]).copy()

for c in CAT_FEATURES:
    model_df[c] = model_df[c].astype("category")

# NaN w feature’ach: na start twardo drop (czytelniej). Potem można robić imputację.
model_df = model_df.dropna(subset=NUM_FEATURES)

model_df.shape

(3490, 25)

In [20]:
def demand_profile(frame: pd.DataFrame):
    g = (
        frame.groupby(["product_name", "country"], observed=True)
        .agg(
            n_rows=(TARGET_RAW, "size"),
            n_series=("sku", "nunique"),
            mean=(TARGET_RAW, "mean"),
            median=(TARGET_RAW, "median"),
            p90=(TARGET_RAW, lambda s: np.nanpercentile(s, 90)),
            max=(TARGET_RAW, "max"),
            zero_share=(TARGET_RAW, lambda s: float((s==0).mean())),
            cv=(TARGET_RAW, lambda s: float(np.nanstd(s) / (np.nanmean(s)+1e-9))),
        )
        .reset_index()
    )
    return g

profile = demand_profile(model_df)
profile.sort_values(["product_name","country"]).head(30)

,product_name,country,n_rows,n_series,mean,median,p90,max,zero_share,cv
0,CERVEZA 30L,Spain,300,1,1592.125000,1600.5,1972.300,3656.000,0.003333,0.229326
1,CHOC QT 05 MC,Portugal,298,1,24.222631,20.0,47.000,84.000,0.000000,0.641389
2,CLAM,Sweden,174,1,272.862069,274.0,328.700,382.000,0.000000,0.184653
3,CONO,Spain,295,1,204.584315,199.0,334.800,432.400,0.003390,0.444734
4,Chipsy PAPRYKA,Poland,129,1,56.262016,40.0,117.800,195.000,0.000000,0.725728
5,Cinnamon,Romania,230,1,3.434783,2.0,7.000,20.000,0.004348,0.958036
6,Coca Cola 20 L BIB,Germany,150,1,9178.173333,9144.5,10415.400,11981.000,0.000000,0.111781
7,Coffee,Romania,66,1,213.540439,207.5,261.917,356.000,0.000000,0.188607
8,GAZPACHO (17783-000),Spain,134,1,2127.529224,2012.0,3104.000,3574.667,0.000000,0.277334
9,LEITE SUNDAE,Portugal,193,1,7274.833767,7160.0,9977.000,11749.000,0.000000,0.289630


In [21]:
profile.shape

(15, 10)

In [22]:
profile.shape
profile.sort_values("mean", ascending=False).head(20)

,product_name,country,n_rows,n_series,mean,median,p90,max,zero_share,cv
6,Coca Cola 20 L BIB,Germany,150,1,9178.173333,9144.5,10415.400,11981.000,0.000000,0.111781
14,TLUSZCZ,Poland,305,1,7721.967213,7700.0,9353.600,17629.000,0.000000,0.196670
9,LEITE SUNDAE,Portugal,193,1,7274.833767,7160.0,9977.000,11749.000,0.000000,0.289630
12,SHAKE LOWFAT,Sweden,304,1,3381.009868,3238.5,4553.000,6135.000,0.000000,0.242075
10,Milk,Sweden,304,1,2458.759043,2367.0,3054.700,4593.000,0.000000,0.165505
8,GAZPACHO (17783-000),Spain,134,1,2127.529224,2012.0,3104.000,3574.667,0.000000,0.277334
0,CERVEZA 30L,Spain,300,1,1592.125000,1600.5,1972.300,3656.000,0.003333,0.229326
13,SUNDAE,Romania,304,1,824.637664,751.5,1424.200,2827.000,0.000000,0.529065
11,PLACINTA VISINE,Romania,304,1,469.708895,406.0,826.000,1708.000,0.000000,0.566522
2,CLAM,Sweden,174,1,272.862069,274.0,328.700,382.000,0.000000,0.184653


In [23]:
profile.sort_values("cv", ascending=False).head(20)

,product_name,country,n_rows,n_series,mean,median,p90,max,zero_share,cv
5,Cinnamon,Romania,230,1,3.434783,2.0,7.000,20.000,0.004348,0.958036
4,Chipsy PAPRYKA,Poland,129,1,56.262016,40.0,117.800,195.000,0.000000,0.725728
1,CHOC QT 05 MC,Portugal,298,1,24.222631,20.0,47.000,84.000,0.000000,0.641389
11,PLACINTA VISINE,Romania,304,1,469.708895,406.0,826.000,1708.000,0.000000,0.566522
13,SUNDAE,Romania,304,1,824.637664,751.5,1424.200,2827.000,0.000000,0.529065
3,CONO,Spain,295,1,204.584315,199.0,334.800,432.400,0.003390,0.444734
9,LEITE SUNDAE,Portugal,193,1,7274.833767,7160.0,9977.000,11749.000,0.000000,0.289630
8,GAZPACHO (17783-000),Spain,134,1,2127.529224,2012.0,3104.000,3574.667,0.000000,0.277334
12,SHAKE LOWFAT,Sweden,304,1,3381.009868,3238.5,4553.000,6135.000,0.000000,0.242075
0,CERVEZA 30L,Spain,300,1,1592.125000,1600.5,1972.300,3656.000,0.003333,0.229326


In [24]:
# Segment A: wysoka skala, niska/średnia zmienność
SEG_A_PRODUCTS = profile.loc[
    (profile["mean"] >= 1500) & (profile["cv"] <= 0.3),
    "product_name"
].unique().tolist()

SEG_A_PRODUCTS

['CERVEZA 30L',
 'Coca Cola 20 L BIB',
 'GAZPACHO (17783-000)',
 'LEITE SUNDAE',
 'Milk',
 'SHAKE LOWFAT',
 'TLUSZCZ']

In [25]:
SEG_A_PRODUCTS = [
    "Coca Cola 20 L BIB",
    "TLUSZCZ",
    "LEITE SUNDAE",
    "SHAKE LOWFAT",
    "Milk",
    "CERVEZA 30L",
]

Tworzymy zbiór danych tylko dla Segmentu A.
Model będzie uczony na wielu seriach jednocześnie,
ale tylko o podobnej charakterystyce popytu.

Kraj i produkt pozostają cechami kategorycznymi,
co pozwala modelowi uczyć się różnic między nimi.

In [26]:
df_A = model_df[model_df["product_name"].isin(SEG_A_PRODUCTS)].copy()

df_A.shape, df_A[["product_name", "country"]].drop_duplicates()

((1556, 25),
             product_name   country
 0     Coca Cola 20 L BIB   Germany
 150              TLUSZCZ    Poland
 585         LEITE SUNDAE  Portugal
 2479         CERVEZA 30L     Spain
 2785        SHAKE LOWFAT    Sweden
 3273                Milk    Sweden)

Stosujemy prosty podział czasowy oparty na dacie:
- 80% najstarszych obserwacji → trening,
- 20% najnowszych → walidacja.

Ten sam split jest używany dla wszystkich serii
w segmencie, co zapewnia porównywalność wyników.

In [27]:
SPLIT_DATE_A = df_A["week_start"].quantile(0.8)

train_A = df_A[df_A["week_start"] <= SPLIT_DATE_A].copy()
valid_A = df_A[df_A["week_start"] > SPLIT_DATE_A].copy()

train_A.shape, valid_A.shape

((1246, 25), (310, 25))

Model:
- LightGBM Regressor,
- trenowany na log1p(demand),
- wspólny dla wszystkich serii w segmencie,
- bez agresywnej regularizacji (najpierw stabilność).

Na tym etapie nie stroimy hiperparametrów —
najpierw sprawdzamy, czy ML w ogóle wygrywa z baseline.

In [28]:
model_A = LGBMRegressor(
    objective="regression",
    learning_rate=0.05,
    n_estimators=2000,
    num_leaves=15,
    min_data_in_leaf=10,
    feature_fraction=0.8,
    subsample=0.8,
    random_state=42,
    verbosity=-1,
)

model_A.fit(
    train_A[CAT_FEATURES + NUM_FEATURES],
    train_A[TARGET_LOG],
    categorical_feature=CAT_FEATURES,
    eval_set=[(valid_A[CAT_FEATURES + NUM_FEATURES], valid_A[TARGET_LOG])],
    eval_metric="l1",
    callbacks=[lgb.early_stopping(50, verbose=False)],
)

,boosting_type,'gbdt'
,num_leaves,15
,max_depth,-1
,learning_rate,0.05
,n_estimators,2000
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


Predykcje cofamy do oryginalnej skali popytu
i liczymy WAPE, aby wynik był porównywalny
z baseline’ami biznesowymi.

In [31]:
valid_A = valid_A.copy()
valid_A["y_pred_log"] = model_A.predict(valid_A[CAT_FEATURES + NUM_FEATURES])
valid_A["y_pred"] = np.expm1(valid_A["y_pred_log"])

wape_A = wape(valid_A[TARGET_RAW].values, valid_A["y_pred"].values)
wape_A

np.float64(0.10391777206864156)

In [32]:
def eval_metrics(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

    mape = np.nan
    mask = y_true != 0
    if mask.any():
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

    bias = np.mean(y_pred - y_true)

    return {
        "WAPE": wape(y_true, y_pred),
        "MAE": mae,
        "RMSE": rmse,
        "MAPE": mape,
        "BIAS": bias,
    }

Segment B obejmuje serie o średniej skali popytu
oraz umiarkowanej zmienności.

Są to produkty:
- o wyraźnym sygnale sezonowym,
- z większym szumem niż Segment A,
- ale nadal bez silnej intermittency.

Celem Segmentu B nie jest osiągnięcie tak niskiego WAPE
jak w Segmencie A, lecz sprawdzenie, czy model ML
nadal systematycznie poprawia baseline.

Segment B (Mid scale / Mid CV):

- skala popytu: średnia,
- zmienność (CV): średnia,
- serie bardziej „nerwowe” niż Segment A,
  ale nadal z sensowną strukturą czasową.

Segment B to kluczowy test:
jeżeli ML wygrywa tutaj z baseline,
to podejście segmentowe jest uzasadnione.

In [33]:
# Segment B: średnia skala, średnia zmienność
SEG_B_PRODUCTS = profile.loc[
    (profile["mean"] < 1500) & (profile["mean"] >= 200) &
    (profile["cv"] > 0.3) & (profile["cv"] <= 0.7),
    "product_name"
].unique().tolist()

SEG_B_PRODUCTS

['CONO', 'PLACINTA VISINE', 'SUNDAE']

In [41]:
SEG_B_PRODUCTS = [
    "GAZPACHO (17783-000)",
    "SUNDAE",            
    "CONO",
    "PLACINTA VISINE",
]

Tworzymy zbiór danych tylko dla Segmentu B.

Model:
- uczy się na wielu seriach jednocześnie,
- wykorzystuje te same cechy co w Segmencie A,
- różnice produktowe i krajowe są kodowane
  jako cechy kategoryczne.

In [37]:
df_B = model_df[model_df["product_name"].isin(SEG_B_PRODUCTS)].copy()

df_B.shape, df_B[["product_name", "country"]].drop_duplicates()

((1037, 25),
               product_name  country
 1084                SUNDAE  Romania
 1389       PLACINTA VISINE  Romania
 2039                  CONO    Spain
 2345  GAZPACHO (17783-000)    Spain)

Stosujemy identyczny podział czasowy jak w Segmencie A:
- 80% najstarszych obserwacji → trening,
- 20% najnowszych → walidacja.

Dzięki temu wyniki segmentów są porównywalne.

In [38]:
SPLIT_DATE_B = df_B["week_start"].quantile(0.8)

train_B = df_B[df_B["week_start"] <= SPLIT_DATE_B].copy()
valid_B = df_B[df_B["week_start"] > SPLIT_DATE_B].copy()

train_B.shape, valid_B.shape

((831, 25), (206, 25))

Używamy dokładnie tego samego modelu i parametrów
co w Segmencie A.

Nie zmieniamy architektury:
- różnice w wynikach wynikają z danych,
- nie z „lepszego” lub „gorszego” modelu.

In [39]:
model_B = LGBMRegressor(
    objective="regression",
    learning_rate=0.05,
    n_estimators=2000,
    num_leaves=15,
    min_data_in_leaf=10,
    feature_fraction=0.8,
    subsample=0.8,
    random_state=42,
    verbosity=-1,
)

model_B.fit(
    train_B[CAT_FEATURES + NUM_FEATURES],
    train_B[TARGET_LOG],
    categorical_feature=CAT_FEATURES,
    eval_set=[(valid_B[CAT_FEATURES + NUM_FEATURES], valid_B[TARGET_LOG])],
    eval_metric="l1",
    callbacks=[lgb.early_stopping(50, verbose=False)],
)

,boosting_type,'gbdt'
,num_leaves,15
,max_depth,-1
,learning_rate,0.05
,n_estimators,2000
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


Predykcje cofamy do oryginalnej skali popytu
i liczymy WAPE, aby porównać ML z baseline’ami.

W Segmencie B oczekujemy:
- wyższego WAPE niż w Segmencie A,
- ale nadal konkurencyjnego lub lepszego
  wyniku względem baseline.

In [40]:
valid_B = valid_B.copy()
valid_B["y_pred_log"] = model_B.predict(valid_B[CAT_FEATURES + NUM_FEATURES])
valid_B["y_pred"] = np.expm1(valid_B["y_pred_log"])

wape_B = wape(valid_B[TARGET_RAW].values, valid_B["y_pred"].values)
wape_B

np.float64(0.1771033314858404)

In [42]:
# porównanie ML vs baseline – Segment B

valid_series_B = valid_B[["country", "sku"]].drop_duplicates()

baseline_valid_B = valid_series_B.merge(
    baseline_ref[["country", "sku", "baseline_wape"]],
    on=["country", "sku"],
    how="left",
)

baseline_wape_B = baseline_valid_B["baseline_wape"].median()

wape_B, baseline_wape_B

(np.float64(0.1771033314858404), 0.24558573049597288)

Segment B:

Model ML osiąga WAPE ≈ 17.7%,
co jest wyraźnie lepszym wynikiem
niż medianowy baseline (≈ 24.6%).

Segment B zostaje objęty modelem ML.
Dalsza segmentacja ani tuning
nie są uzasadnione.

Segment C obejmuje serie o niskiej skali popytu
i wysokiej zmienności.

Są to produkty:
- o małych wolumenach,
- z dużym szumem względnym,
- często bez wyraźnej sezonowości.

Dla Segmentu C:
- NIE oczekujemy niskiego WAPE,
- celem jest sprawdzenie, czy ML
  nie przegrywa znacząco z baseline.

Jeżeli ML ≈ baseline → baseline zostaje.
Jeżeli ML wygrywa → traktujemy to jako bonus.

Segment C (Low scale / High CV):

- niska skala popytu,
- wysoka zmienność (CV),
- największe ryzyko overfittingu.

To jest segment, w którym baseline
bardzo często jest wystarczający.

In [43]:
# Segment C: niska skala, wysoka zmienność
SEG_C_PRODUCTS = profile.loc[
    (profile["mean"] < 200) & (profile["cv"] > 0.7),
    "product_name"
].unique().tolist()

SEG_C_PRODUCTS

['Chipsy PAPRYKA', 'Cinnamon']

In [44]:
SEG_C_PRODUCTS = [
    "Cinnamon",
    "Chipsy PAPRYKA",
    "CHOC QT 05 MC",
]

Tworzymy zbiór danych dla Segmentu C.

Liczba serii jest niewielka,
dlatego szczególnie ważne jest,
aby nie komplikować modelu
ani nie stroić parametrów.

In [45]:
df_C = model_df[model_df["product_name"].isin(SEG_C_PRODUCTS)].copy()

df_C.shape, df_C[["product_name", "country"]].drop_duplicates()

((657, 25),
         product_name   country
 456   Chipsy PAPRYKA    Poland
 778    CHOC QT 05 MC  Portugal
 1761        Cinnamon   Romania)

Stosujemy identyczny podział czasowy
jak w Segmentach A i B (80/20).

Dzięki temu wszystkie segmenty
są porównywalne.

In [46]:
SPLIT_DATE_C = df_C["week_start"].quantile(0.8)

train_C = df_C[df_C["week_start"] <= SPLIT_DATE_C].copy()
valid_C = df_C[df_C["week_start"] > SPLIT_DATE_C].copy()

train_C.shape, valid_C.shape

((526, 25), (131, 25))

Używamy dokładnie tego samego modelu
i parametrów co w Segmentach A i B.

W Segmencie C NIE:
- zwiększamy złożoności,
- nie stroimy hiperparametrów,
- nie optymalizujemy agresywnie.

To test generalizacji, nie wydajności.

In [47]:
model_C = LGBMRegressor(
    objective="regression",
    learning_rate=0.05,
    n_estimators=2000,
    num_leaves=15,
    min_data_in_leaf=10,
    feature_fraction=0.8,
    subsample=0.8,
    random_state=42,
    verbosity=-1,
)

model_C.fit(
    train_C[CAT_FEATURES + NUM_FEATURES],
    train_C[TARGET_LOG],
    categorical_feature=CAT_FEATURES,
    eval_set=[(valid_C[CAT_FEATURES + NUM_FEATURES], valid_C[TARGET_LOG])],
    eval_metric="l1",
    callbacks=[lgb.early_stopping(50, verbose=False)],
)

,boosting_type,'gbdt'
,num_leaves,15
,max_depth,-1
,learning_rate,0.05
,n_estimators,2000
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


Predykcje cofamy do oryginalnej skali popytu
i liczymy WAPE.

W Segmencie C:
- wysoki WAPE jest oczekiwany,
- decyzja zależy wyłącznie od porównania
  z baseline.

In [48]:
valid_C = valid_C.copy()
valid_C["y_pred_log"] = model_C.predict(valid_C[CAT_FEATURES + NUM_FEATURES])
valid_C["y_pred"] = np.expm1(valid_C["y_pred_log"])

wape_C = wape(valid_C[TARGET_RAW].values, valid_C["y_pred"].values)
wape_C

np.float64(0.33716779356082416)

In [49]:
valid_series_C = valid_C[["country", "sku"]].drop_duplicates()

baseline_valid_C = valid_series_C.merge(
    baseline_ref[["country", "sku", "baseline_wape"]],
    on=["country", "sku"],
    how="left",
)

baseline_wape_C = baseline_valid_C["baseline_wape"].median()

wape_C, baseline_wape_C

(np.float64(0.33716779356082416), 0.5146627565982405)